# 实体对齐

In [2]:
include("../src/CCKS-mKGQA.jl")
include("../src/loaddata/extractdata.jl")
include("../src/loaddata/translatedata.jl")
include("../src/process/ettalign.jl")
mkpath.("EA_data/" .* (["", "ILLs_1", "ILLs_2", "ILLs_3", "ILLs_4", "ILLs_official", "ILLs_wiki"]))

7-element Vector{String}:
 "EA_data"
 "EA_data/ILLs_1"
 "EA_data/ILLs_2"
 "EA_data/ILLs_3"
 "EA_data/ILLs_4"
 "EA_data/ILLs_official"
 "EA_data/ILLs_wiki"

### 初始化

In [10]:
illpath = "ILLs_1"
illpath = "ILLs_2"
illpath = "ILLs_3"
illpath = "ILLs_4"
illpath = "ILLs_official"
illpath = "ILLs_wiki"
include("../src/loaddata/illsdata.jl");

In [11]:
# ILLs => 对齐字典
path = "ILLs_3"
newILLs = readtuples("EA_data/$path/ILLs_predict_zh_en.txt"; size=2);

In [20]:
path = "ILLs_wiki"
newILLs = wiki_ILLs;

### 生成文件

In [11]:
filter!(i->i[1] ∈ wordset, newILLs)
mt_newILLs = [getindex.(Ref(dict_words), pair) for pair in newILLs]
filter!(i->i[1]!=i[2], unique!(mt_newILLs))
println("对齐数目：\t", length.([newILLs, mt_newILLs])) # 13055 => 9802
dict_ILLs = ill2dict(mt_newILLs)

# 翻译三元组
mt_triples = triple_byMT.(triples)

# 对齐三元组
triple_byills(triple) = triple_byills(triple...)
triple_byills(sub, rel, obj) = (get(dict_ILLs, sub, sub), rel, get(dict_ILLs, obj, obj))
ill_triples = unique!(triple_byills.(mt_triples))
println("元组数目：\t", length.([triples, ill_triples]))

# 验证集
ill_valid_ners = [get(dict_ILLs, ner, ner) for ner in last.(mt_valid_ques_ner)]
ill_valid_ques = first.(mt_valid_ques_ner)

# 训练集
ill_train_ners = [get(dict_ILLs, ner, ner) for ner in getindex.(mt_train_ques_rels, 2)]
ill_train_ques = mt_train_ques;
ill_train_sols = [triple_byills.(triple_byMT.(getindex.(sols, Ref(2:4)))) for sols in train_sols]
ill_train_rels = [getindex.(sols, 2) for sols in ill_train_sols];

对齐数目：	[42072, 29655]
元组数目：	[267399, 261590]


### 测试

In [13]:
edges = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (sub, rel, obj) in ill_triples
    push!(edges[sub], (rel, obj))
end

subs = unique!(first.(ill_triples))
# 第一跳
jumps = Dict(sub => [length(edges[sub])] for sub in subs)
# 第二跳
for sub in subs
    push!(jumps[sub], sum(haskey(jumps, obj) && jumps[obj][1] for (rel, obj) in edges[sub]))
end
# 第三跳
for sub in subs
    push!(jumps[sub], sum(haskey(jumps, obj) && jumps[obj][2] for (rel, obj) in edges[sub]))
end

In [60]:
# ILLs 3
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

[0, 37283, 48256]


(61, 137)

In [9]:
# 官方 ILLs
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

[0, 52268, 67437]


(156, 352)

In [25]:
# ILLs 1
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

[0, 44396, 57285]


(80, 209)

In [10]:
# ILLs 2
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

[0, 47455, 61439]


(98, 263)

In [14]:
# wiki ILLs
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

[0, 38070, 49921]


(6, 103)

### 保存

In [15]:
open("EA_data/$path/triples.txt", "w") do io
    for triple in ill_triples
        println(io, join(triple, '\t'))
    end
end

open("EA_data/$path/train_ques.txt", "w") do io
    for (que, ner, rels) in zip(ill_train_ques, ill_train_ners, ill_train_rels)
        que = replace(que, r", (a|an) .*?[,?]" => "")
        println(io, que, '\t', ner, '\t', join(rels, '\t'))
    end
end

open("EA_data/$path/train_data.txt", "w") do io
    for (que, ner, sols) in zip(ill_train_ques, ill_train_ners, ill_train_sols)
        que = replace(que, r", (a|an) .*?[,?]" => "")
        println(io, que, '\t', ner)
        for sol in sols
            println(io, join(sol, '\t'))
        end
        println(io)
    end
end

open("EA_data/$path/valid_ques.txt", "w") do io
    for (que, ner) in zip(ill_valid_ques, ill_valid_ners)
        que = replace(que, r", (a|an) .*?[,?]" => "")
        println(io, que, '\t', ner)
    end
end

LoadError: SystemError: opening file "EA_data/path/triples.txt": 没有那个文件或目录